In [1]:
%cd ../
## Autorreload all the files
%load_ext autoreload
%autoreload 2

import numpy as np
import matplotlib.pyplot as plt
import torch
from torch import nn
from scipy.io.arff import loadarff
import pandas as pd
from sklearn.model_selection import train_test_split
import seaborn as sns
from sklearn.metrics import mean_squared_error, accuracy_score#, root_mean_squared_error

import importlib
from Scripts.KANLayer import KANLayer
from Scripts.KAN import KAN
from Scripts import spline
from Scripts import utils
from Scripts.CustomELU import CustomELU
from Scripts.grid_search import grid_search

print("Device", torch.cuda.is_available())

f:\MonoKAN


ModuleNotFoundError: No module named 'spline'